# Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
plt.rcParams['font.size'] = 16

# Read data

In [ ]:
data = pd.read_csv('/home/asberk/data/1-Donaldson/AllData.csv')

In [ ]:
data.head()

In [ ]:
# from pprint import pprint
# pprint(data.columns.tolist())

## Cleaning up the junk

In [ ]:
def checkColumn(df, colNum):
    """
    Used in throwAwayUnchanged
    """
    return np.all(df.iloc[0, colNum] == df.iloc[1:, colNum])


def throwAwayUnchanged(df):
    """
    Made specifically for the data we were given for the Midvale project. 
    Could, however, prove useful on subsetted-by-group data...
    This function throws away columns that are the same in every entry
    """
    idxUnhelpful = [j for j in range(df.columns.size)
                    if checkColumn(df, j)]
    df = df.drop(df.columns[idxUnhelpful], axis=1)
    return df


def throwAwayBizarre(df):
    """
    Throws away rows where TotalBytes is negative 
    (because this doesn't make sense).
    """
    df = df.loc[df['TotalBytes'] >= 0]
    return df


def removeUnwanted(data):
    """
    Made specifically for the data we were given for the Midvale project. 
    """
    # Don't worry about High Performance mode for this task
    data = data.groupby('Mode').get_group(0)
    # Flicker is not useful for prediction
    data = data.drop('Flicker', axis=1)
    # We will throw away columns that are all the same
    # (On `data`, this gets rid of Sharpening, 
    #  WaitSeconds and Status)
    data = throwAwayUnchanged(data)
    data = throwAwayBizarre(data)
    data = data.drop('Index', axis=1)
    return data


def fixMiscValues(df):
    """
    Made specifically for the data we were given for the Midvale project. 
    """
    df = df.fillna({'TertiaryResolution': 'NaN'})
    df = df.replace('-', value=0)
    df['SecondaryBitsPerSecond'] = df['SecondaryBitsPerSecond'].astype(np.float64)
    df['TertiaryBitsPerSecond'] = df['TertiaryBitsPerSecond'].astype(np.float64)
    return df


def preProcess(df):
    """
    Made specifically for the data we were given for the Midvale project
    """
    df = removeUnwanted(df)
    df = fixMiscValues(df)
    return df

In [ ]:
data = preProcess(data)

# Make a subset of the data for a simpler time

First we have to figure out the subset...

Roger recommended sticking with `Test == Base` and a single camera. Let's choose the camera with the most observations.

In [ ]:
Base_gb_CameraName = data.loc[data['Test']=='Base'].groupby(['CameraName'])
CameraName_highestCountOf_Base = Base_gb_CameraName.count()['Test'].argmax()
data_A3Base = Base_gb_CameraName.get_group(CameraName_highestCountOf_Base)
data_A3Base = data_A3Base.drop(['CameraName', 'Test'], axis=1)

In [ ]:
data_A3Base.head()

# Data exploration

## Histogram of continuous

In [ ]:
logBytes = data.loc[:, ['TotalBytes', 'PrimaryBitsPerSecond', 'SecondaryBitsPerSecond', 'TertiaryBitsPerSecond']]
logBytes = logBytes.replace(0., np.nan).apply(lambda x: np.log(x)).dropna(how='all')
logBytes.hist();

In [ ]:
def logTransformColumn(df, colname):
    """
    Tailor-made for the Midvale data. 
    log-transforms the columns pertaining to bit-rate.
    """
    logBytes = data[colname]
    logBytes = logBytes.replace(0., np.nan).apply(lambda x: np.log(x))
    logBytes = logBytes.dropna(how='all')
    return df.assign(**{'log'+logBytes.name: logBytes})

def logTransformBytes(df):
    for columnName in ['TotalBytes', 'PrimaryBitsPerSecond', 
                       'SecondaryBitsPerSecond', 'TertiaryBitsPerSecond']:
        df = logTransformColumn(df, columnName)
    return df

In [ ]:
data = logTransformBytes(data)
data_A3Base = logTransformBytes(data_A3Base)

In [ ]:
data.head()

In [ ]:
data_A3Base.head()

## Histogram of categoricals

In [ ]:
def hist_colVals(X, **kwargs):
    """
    X : a categorical column of a data frame
    """
    # Check if not categorical
    #
    #
    # get value counts
    vc = X.value_counts()
    n = vc.shape[0]
    xrange = np.arange(n)
    plt.bar(xrange, vc.values, **kwargs)
    plt.xticks(xrange, vc.index.tolist(), rotation=90)
    return
    
    

In [ ]:
# make a histogram of these
# (these are the *useful* categories for CamerName:A3;Test:Base)
categs = ['PrimaryResolution', 'SecondaryResolution', 
          'Keyframe', 'ImageRate', 'Quality',
          'Detail', 'Motion']

C = len(categs)
ncols = 4
nrows = np.int(np.ceil(C/5))
figwidth = 20
figheight = np.int(np.min([np.ceil(20/ncols*nrows), 20]))

fig, axes = plt.subplots(nrows, ncols, figsize=(figwidth, figheight))
fig.subplots_adjust(hspace=.75)

for j, categ in enumerate(categs):
    plt.subplot(nrows, ncols, j+1)
    hist_colVals(data_A3Base[categ])
    plt.xlabel(categ, labelpad=20)
for j in range(C, nrows*ncols):
    plt.subplot(nrows, ncols, j+1)
    plt.axis('off')

## Finding correlations with PrimaryBitsPerSecond

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
data_A3Base['logPrimaryBitsPerSecond'].hist(bins=1000, cumulative=True)

In [ ]:
scaler = StandardScaler()
qualityResponse = scaler.fit_transform(data_A3Base.loc[:, ['Quality', 'logPrimaryBitsPerSecond']])

In [ ]:
plt.hist2d(qualityResponse[:,0],
           qualityResponse[:,1],
           bins=20);
plt.xlabel('Quality')
plt.ylabel('log(PrimaryBitsPerSecond)')
plt.colorbar();

# A very simple linear regression

In [ ]:
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_A3Base.loc[:,['Quality', 'ImageRate', 'Keyframe']].values,
                                                    data_A3Base['logPrimaryBitsPerSecond'].values)

In [ ]:
en = ElasticNetCV(normalize=True)
en.fit(X_train, y_train)

In [ ]:
en.score(X_test, y_test)

# Encoding features for regression

## Encoding the categoricals

If there are any categoricals, then maybe we should put columns as integers so we can regress on them? 

In [ ]:
def categDF(data):
    categoricals = data.select_dtypes(include=['object'])
    categoricals = categoricals.drop('Message', axis=1)
    return categoricals


In [ ]:
def setUpCategs(data, sparse=False):
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    lb = LabelEncoder()
    oh = OneHotEncoder()
    categoricals = categDF(data)
    categoricals = categoricals.apply(lb.fit_transform)
    categoricals = oh.fit_transform(categoricals)
    if not sparse:
        categoricals = categoricals.toarray()
    return categoricals


In [ ]:
def unencodeOneHotLabelling(ohEnc, oh, lbl):
    return lbl.inverse_transform(oh.active_features_)[np.argmax(ohEnc, axis=-1)]

In [ ]:
categoricals = setUpCategs(data)

In [ ]:
categoricals_A3Base = setUpCategs(data_A3Base)

In [ ]:
categ_df = categDF(data)

In [ ]:
categ_df_A3Base = categDF(data_A3Base)

In [ ]:
categ_df_A3Base.head()

### Can we unscramble categoricals?

In [ ]:
elems = pd.DataFrame(np.array([['cat', 'dog', 'mouse', 'mouse', 'cat', 'guineapig'],['white', 'white', 'white', 'red', 'black', 'orange']]).T, columns=['animal', 'colour'])
elems

In [ ]:
lblDemo = LabelEncoder()
ohDemo = OneHotEncoder()

In [ ]:
lblDemo.fit_transform(elems.values.ravel())

In [ ]:
lblDemo.inverse_transform([1,2,3])

In [ ]:
ohEncodingOfAnimals = ohDemo.fit_transform(np.c_[lblDemo.fit_transform(elems.values.ravel())]).toarray()

In [ ]:
lblDemo.inverse_transform(ohDemo.active_features_)[np.argmax(ohEncodingOfAnimals, axis=-1)]

## Now for the continous

In [ ]:
numerical_names = ['Keyframe', 'ImageRate', 'Quality', 'KbpsLimit', 'CollectSeconds']
numerical = data.filter(items=numerical_names)
numerical_A3Base = data_A3Base.filter(items=numerical_names)

In [ ]:
numerical.values[:5,:]

In [ ]:
numerical_A3Base.values[:5,:]

## Now the response(s)
TotalBytes should never be negative so far as I'm aware, so let's fix this:

In [ ]:
response_names = ['logTotalBytes', 'logPrimaryBitsPerSecond', 'logSecondaryBitsPerSecond', 'logTertiaryBitsPerSecond']
responses = data.filter(items=response_names)
responses_A3Base = data_A3Base.filter(items=response_names)

In [ ]:
responses.values[:5,:]

In [ ]:
responses_A3Base.values[:5,:]

## How to unmuddle...

In [ ]:
def getCategNames(df):
    return np.concatenate([np.unique(df[col].values) for col in df.columns])

In [ ]:
def unencodeOneHot(ohArr, df):
    categNames = getCategNames(df)
    return [categNames[np.where(ohArr)[1]][j::2] for j in range(2)]

In [ ]:
categNames_A3Base = getCategNames(categ_df_A3Base)

In [ ]:
names_A3Base = np.concatenate((numerical_names, categNames_A3Base))

# Building regressors on the data

## Prepare the data

First on a small problem, then on a progressively bigger ones

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X_A3Base = np.hstack((scaler.fit_transform(numerical_A3Base.values), categoricals_A3Base))
y_A3Base = responses_A3Base.values
y_A3Base_pbps = y_A3Base[:, 1]

In [ ]:
nonnanrows = [not np.isnan(y_A3Base_pbps[j]) for j in range(y_A3Base_pbps.shape[0])]

X_nonnan = X_A3Base[nonnanrows, :]
y_nonnan = y_A3Base_pbps[nonnanrows]

In [ ]:
print('Splitting vanilla data into train and test...', end='')
X_train, X_test, y_train, y_test = train_test_split(X_nonnan, y_nonnan)
print('done.')

In [ ]:
print('There were no nan values to remove: {}'.format(X_A3Base.shape == X_nonnan.shape))
print('Number of training examples is {}.'.format(X_nonnan.shape))

In [ ]:
print('Generating interaction features of degree ≤ 2...', end='')
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_nonnan_poly2 = poly.fit_transform(X_nonnan)
print('done.')

print('Size of poly features is {}.'.format(X_nonnan_poly2.shape))

print('Splitting poly2 data into train and test...', end='')
Xp2_train, Xp2_test, yp2_train, yp2_test = train_test_split(X_nonnan_poly2, y_nonnan)
print('done.')

## ElasticNet

In [ ]:
en = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], n_jobs=-1)

### Fitting on data with no interaction terms

In [ ]:
en.fit(X_train, y_train)

In [ ]:
Rsq_vanilla_EN = en.score(X_test, y_test)
print('R^2 = {}'.format(Rsq_vanilla_EN))

In [ ]:
MSE_vanilla_EN = mean_squared_error(y_test, en.predict(X_test))
print('MSE = {}'.format(MSE_vanilla_EN))

### Fitting on data with interaction terms

(**Note:** we do negligibly better with poly-degree=3)

In [ ]:
en.fit(Xp2_train, yp2_train)

In [ ]:
Rsq_poly2_EN = en.score(Xp2_test, yp2_test)
print('R^2 = {}'.format(Rsq_poly2_EN))

In [ ]:
MSE_poly2_EN = mean_squared_error(yp2_test, en.predict(Xp2_test))
print('MSE = {}'.format(MSE_poly2_EN))

Nice! I'm getting an R^2 value of .932, which means I'm explaining about 93 % of the variance for the A3 Camera in the Base Test.

### Notes

* We really want to predict `log(PrimaryBitsPerSecond)`. And we probably want to scale it first — Yes! Scaling the numerical sends the $R^2$ from .72 to .93!
* What are the other variables we want to scale? (See above...)
* Are we allowed to use Quality in our prediction? Yes! 

## Random forest regressors

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor

In [ ]:
xTree = ExtraTreesRegressor(n_jobs=-1)
rf = RandomForestRegressor(n_jobs=-1)
gb = GradientBoostingRegressor()

### Fitting RF on vanilla data

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
gb.score(X_test, y_test)

In [ ]:
mean_squared_error(y_test, gb.predict(X_test))

This isn't fair...

#### Visualizing important features

In [ ]:
idx_importance = np.argsort(gb.feature_importances_)[::-1]

print(np.column_stack((names_A3Base, gb.feature_importances_))[idx_importance, :])

### Fitting RFs on data with degree 2 interactions

In [ ]:
gb.fit(Xp2_train, yp2_train)

In [ ]:
gb.score(Xp2_test, yp2_test)

In [ ]:
mean_squared_error(yp2_test, gb.predict(Xp2_test))

Super unfair...

### Viewing important features

In [ ]:
polyFeatureNames_A3Base = poly.get_feature_names(names_A3Base)

Print those features that were **above $1\sigma$ importance**.

In [ ]:
idx_importance_sorted = gb.feature_importances_.argsort()[::-1]

In [ ]:
th = gb.feature_importances_.mean() + gb.feature_importances_.std()
for j in idx_importance_sorted:
    coef = gb.feature_importances_[j]
    if coef < th:
        break
    else:
        ftrName = polyFeatureNames_A3Base[j]
        coef = gb.feature_importances_[j]
        print(ftrName, end='')
        print(' '*(30-len(ftrName)), end='')
        print(coef)